# OpenAI Gym

In [1]:
import gym

env = gym.make("CartPole-v0")

/Users/yamauchito_satoshi/Documents/env/zero_DeepL4/lib/python3.8/site-packages/gym/envs/registration.py:593: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.warn(
/Users/yamauchito_satoshi/Documents/env/zero_DeepL4/lib/python3.8/site-packages/gym/core.py:329: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/Users/yamauchito_satoshi/Documents/env/zero_DeepL4/lib/python3.8/site-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [2]:
state = env.reset()
print(state)

action_space = env.action_space
print(action_space)

[ 0.00604887 -0.02179821 -0.00839396  0.00828056]
Discrete(2)


In [3]:
action = 0
next_state, reward, done, info = env.step(action)
print(next_state)

[ 0.00561291 -0.21679878 -0.00822834  0.2983033 ]


In [5]:
import numpy as np
import gym


env = gym.make('CartPole-v0')
state = env.reset()
done = False

while not done:
    env.render()
    action = np.random.choice([0, 1])
    next_state, reward, done, info = env.step(action)
env.close()